In [2]:
import cv2
import os
import shutil
from tqdm import tqdm
import numpy as np
import json
import torch
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

In [3]:
def get_frame_from_video(video_name, save_dict, interval):
    video_capture = cv2.VideoCapture(video_name)
    i = 1
    while True:
        success, frame = video_capture.read()
        # print(success)
        if not success:
            break
        if i % interval == 0:
            save_name = os.path.join(save_dict, str(i//interval-1), video_name.split(".mp4")[0].split("/")[-1] + "_" + str(i//interval-1) + ".jpg")
            cv2.imwrite(save_name, frame)
        i += 1

In [3]:
video_path = "/blob/v-yuancwang/DiffAudioImg/VGGSound/data/vggsound/video"
save_dict = "/blob/v-yuancwang/DiffAudioImg/VGGSound/data/vggsound/img_spilt"

In [ ]:
video_list = os.listdir(video_path)

In [29]:
video_list.sort()
print(len(video_list))
print(video_list[:100])

176955
['---g-f_I2yQ_1000_11000.mp4', '--0PQM4-hqg_30000_40000.mp4', '--56QUhyDQM_185000_195000.mp4', '--8puiAGLhs_30000_40000.mp4', '--96EN9NUQM_242000_252000.mp4', '--9O4XZOge4_30000_40000.mp4', '--Aa4M484QM_0_10000.mp4', '--Bu2xe4OSo_430000_440000.mp4', '--CC5pH97q4_33000_43000.mp4', '--CC5pH97q4_4000_14000.mp4', '--DTR0_mIGI_11000_21000.mp4', '--DTR0_mIGI_32000_42000.mp4', '--EquRwM9IQ_8000_18000.mp4', '--FenyW2i_4_0_10000.mp4', '--G-wKyj6JQ_30000_40000.mp4', '--HuBNHXu9I_30000_40000.mp4', '--IOqc-XgYo_0_10000.mp4', '--L3BCCcGEw_400000_410000.mp4', '--Lj4Y_96f0_120000_130000.mp4', '--PlJNEnf-s_288000_298000.mp4', '--Q8wkZvDZE_150000_160000.mp4', '--QVnZXkb_Y_74000_84000.mp4', '--QVnZXkb_Y_98000_108000.mp4', '--R3QLObQ5I_319000_329000.mp4', '--SQyOb8eS0_30000_40000.mp4', '--SvivLlKLU_137000_147000.mp4', '--SvivLlKLU_662000_672000.mp4', '--TKJIv9aY4_210000_220000.mp4', '--TKJIv9aY4_282000_292000.mp4', '--U7joUcTCo_0_10000.mp4', '--UE437qBIo_1000_11000.mp4', '--UQXuZfbFY_44000_54000.m

In [4]:
# img_list_0 = os.listdir(os.path.join(save_dict, "0"))
# img_list_0.sort()
# img_list_1 = os.listdir(os.path.join(save_dict, "1"))
# img_list_1.sort()
img_list_2 = os.listdir(os.path.join(save_dict, "2"))
img_list_2.sort()
# img_list_3 = os.listdir(os.path.join(save_dict, "3"))
# img_list_3.sort()
# img_list_4 = os.listdir(os.path.join(save_dict, "4"))
# img_list_4.sort()

In [5]:
# print(len(img_list_0), img_list_0[:10])
# print(len(img_list_1), img_list_1[:10])
print(len(img_list_2), img_list_2[:10])
# print(len(img_list_3), img_list_3[:10])
# print(len(img_list_4), img_list_4[:10])

176858 ['--DTR0_mIGI_11000_21000_2.jpg', '--DTR0_mIGI_32000_42000_2.jpg', '--EquRwM9IQ_8000_18000_2.jpg', '--FenyW2i_4_0_10000_2.jpg', '--G-wKyj6JQ_30000_40000_2.jpg', '--HuBNHXu9I_30000_40000_2.jpg', '--IOqc-XgYo_0_10000_2.jpg', '--L3BCCcGEw_400000_410000_2.jpg', '--Lj4Y_96f0_120000_130000_2.jpg', '--PlJNEnf-s_288000_298000_2.jpg']


In [7]:
mel_path = "/blob/v-yuancwang/DiffAudioImg/VGGSound/data/vggsound/mel"
mel_list = os.listdir(mel_path)
mel_list.sort()

In [8]:
print(len(mel_list))
print(mel_list[:10])

177045
['---g-f_I2yQ_1000_11000.npy', '--0PQM4-hqg_30000_40000.npy', '--56QUhyDQM_185000_195000.npy', '--8puiAGLhs_30000_40000.npy', '--96EN9NUQM_242000_252000.npy', '--9O4XZOge4_30000_40000.npy', '--Aa4M484QM_0_10000.npy', '--Bu2xe4OSo_430000_440000.npy', '--CC5pH97q4_33000_43000.npy', '--CC5pH97q4_4000_14000.npy']


In [12]:
# img_set_0 = set(img_list_0)
# img_set_1 = set(img_list_1)
img_set_2 = set(img_list_2)
# img_set_3 = set(img_list_3)
# img_set_4 = set(img_list_4)

In [13]:
with open("/home/v-yuancwang/DiffAudioImg/VGGSound/data/vggsound.csv") as f:
    lines = f.readlines()
vgg_infos = {}
for line in lines:
    line = line.split(",")
    vgg_infos[line[0]+"_"+str(int(line[1])*1000)+"_"+str((int(line[1])+10)*1000)] = {"text":line[2], "train": line[3].replace("\n", "")}
len(vgg_infos)

199467

In [14]:
vgg_train_lists_0 = []
vgg_train_lists_1 = []
vgg_train_lists_2 = []
vgg_train_lists_3 = []
vgg_train_lists_4 = []

for mel_file in tqdm(mel_list):
    if vgg_infos[mel_file.replace(".npy", "")]["train"] == "test":
        continue
    # img_file = mel_file.replace(".npy", "") + "_0.jpg"
    # if img_file in img_set_0:
    #     vgg_train_lists_0.append({"image": os.path.join("0", img_file),
    #                             "conditioning_mel": os.path.join(mel_file),
    #                             "text": vgg_infos[mel_file.replace(".npy", "")]["text"].replace("\"", "", 4)})
    # img_file = mel_file.replace(".npy", "") + "_1.jpg"
    # if img_file in img_set_1:
    #     vgg_train_lists_1.append({"image": os.path.join("1", img_file),
    #                             "conditioning_mel": os.path.join(mel_file),
    #                             "text": vgg_infos[mel_file.replace(".npy", "")]["text"].replace("\"", "", 4)})
    img_file = mel_file.replace(".npy", "") + "_2.jpg"
    if img_file in img_set_2:
        vgg_train_lists_2.append({"image": os.path.join("2", img_file),
                                "conditioning_mel": os.path.join(mel_file),
                                "text": vgg_infos[mel_file.replace(".npy", "")]["text"].replace("\"", "", 4)})
    # img_file = mel_file.replace(".npy", "") + "_3.jpg"
    # if img_file in img_set_3:
    #     vgg_train_lists_3.append({"image": os.path.join("3", img_file),
    #                             "conditioning_mel": os.path.join(mel_file),
    #                             "text": vgg_infos[mel_file.replace(".npy", "")]["text"].replace("\"", "", 4)})
    # img_file = mel_file.replace(".npy", "") + "_4.jpg"
    # if img_file in img_set_4:
    #     vgg_train_lists_4.append({"image": os.path.join("4", img_file),
    #                             "conditioning_mel": os.path.join(mel_file),
    #                             "text": vgg_infos[mel_file.replace(".npy", "")]["text"].replace("\"", "", 4)})

100%|██████████| 177045/177045 [00:00<00:00, 227013.53it/s]


In [31]:
print(len(vgg_train_lists_0[:]))
with open("/home/v-yuancwang/DiffAudioImg/metadata/vgg_train_0.json", "w") as f:
    json.dump(vgg_train_lists_0, fp=f)
print(len(vgg_train_lists_1[:]))
with open("/home/v-yuancwang/DiffAudioImg/metadata/vgg_train_1.json", "w") as f:
    json.dump(vgg_train_lists_1, fp=f)
print(len(vgg_train_lists_2[:]))
with open("/home/v-yuancwang/DiffAudioImg/metadata/vgg_train_2.json", "w") as f:
    json.dump(vgg_train_lists_2, fp=f)
print(len(vgg_train_lists_3[:]))
with open("/home/v-yuancwang/DiffAudioImg/metadata/vgg_train_3.json", "w") as f:
    json.dump(vgg_train_lists_3, fp=f)
print(len(vgg_train_lists_4[:]))
with open("/home/v-yuancwang/DiffAudioImg/metadata/vgg_train_4.json", "w") as f:
    json.dump(vgg_train_lists_4, fp=f)

164369
164353
164326
164251
160863


In [ ]:
test_id = 1
image_test = Image.open(vgg_train_lists_2[test_id]["image"])
image_test

In [103]:
import IPython

In [104]:
mel_test = vgg_train_lists_2[test_id]["conditioning_mel"]
wav_test = mel_test.replace(".npy", ".wav").replace("/mel/", "/wav/")
IPython.display.Audio(wav_test)

In [ ]:
# for video_name in tqdm(video_list[:2000]):
#     video_name = os.path.join(video_path, video_name)
#     get_frame_from_video(video_name, save_dict, 10)